In [1]:
STOP = 0x00
ADD = 0x01
MUL = 0x02
SUB = 0x03
DIV = 0x04
SDIV = 0x05
MOD = 0x06
SMOD = 0x07
ADDMOD = 0x08
MULMOD = 0x09
EXP = 0x0A
SIGNEXTEND = 0x0B
LT = 0x10
GT = 0x11
SLT = 0x12
SGT = 0x13
EQ = 0x14
ISZERO = 0x15
AND = 0x16
OR = 0x17
XOR = 0x18
NOT = 0x19
BYTE = 0x1A
SHL = 0x1B
SHR = 0x1C
SAR = 0x1D
PUSH0 = 0x5F
PUSH1 = 0x60
PUSH32 = 0x7F
POP = 0x50
MLOAD = 0x51
MSTORE = 0x52
MSTORE8 = 0x53
SLOAD = 0x54
SSTORE = 0x55
JUMP = 0x56
JUMPI = 0x57
PC = 0x58
MSIZE = 0x59
JUMPDEST = 0x5B

class StopException(Exception):
    pass

class EVM:
    def __init__(self, code):
        self.code = code            # initialize bytecodes, byte object
        self.pc = 0                 # initialize program counter to 0
        self.stack = []             # initialize stack to null
        self.memory = bytearray()   # initialize memory to null
        self.storage = {}           # initialize storage to null dictionary
        self.validJumpDest = {}     # declare valid destination of Jump instruction

    def next_instruction(self):
        op = self.code[self.pc]     # Get current instruction
        self.pc += 1                # PC increased by 1
        return op
    
    def findValidJumpDestinations(self):
        # make sure JumpDest is not the parameter of PUSH
        pc = 0

        while pc < len(self.code):
            op = self.code[pc]
            if op == JUMPDEST:
                self.validJumpDest[pc] = True
            elif op >= PUSH1 and op <= PUSH32:
                pc += op - PUSH1 + 1
            pc += 1

    def push(self, size):
        data = self.code[self.pc:self.pc + size]    # get data from code according to size
        value = int.from_bytes(data, 'big')         # transfer bytes to int
        self.stack.append(value)                    # Push onto stack
        self.pc += size                             # pc increased by size long step

    def pop(self):
        if len(self.stack) == 0:
            raise Exception('Stack Underflow')
        return self.stack.pop()                     # pop stack
    
    def add(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        res = (a + b) % (2**256)                    # The addition result needs to be modulo 2^256 to prevent overflow
        self.stack.append(res)

    def mul(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        res = (a * b) % (2**256)                    # The multiplication result needs to be modulo 2^256 to prevent overflow
        self.stack.append(res)
    
    def sub(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        res = (a - b) % (2**256)                    # result needs to be modulo 2^256 to prevent overflow
        self.stack.append(res)

    def div(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        if b == 0:
            res = 0
        else:
            res =  (a // b) % (2**256)
        self.stack.append(res)

    def sdiv(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
    
        if b == 0:
            res = 0
        else:
            # Perform sign-extended division
            if a < 0:
                a += 2**256
            if b < 0:
                b += 2**256
        
            res = a // b
        
            # Make sure the result is 256 bits and has the correct sign
            if res >= 2**255:
                res -= 2**256
    
        self.stack.append(res)

    def mod(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        if b == 0:          # res = (a % b if b != 0 else 0)
            res = 0
        else:
            res = a % b
        self.stack.append(res)

    def smod(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()

        if b == 0:
          res = 0
        else:
            # Convert to a signed integer
            if a >= 2**255:
                a -= 2**256
            if b >= 2**255:
                b -= 2**256

            res = abs(a) % abs(b)
            if a < 0 or b < 0:
                res = -res

            # Ensure that the result is a 256-bit unsigned integer
            res = res % (2**256)

        self.stack.append(res)

    def addmod(self):
        if len(self.stack) < 3:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        n = self.stack.pop()
        res = (a + b) % n if n != 0 else 0
        self.stack.append(res)

    def mulmod(self):
        if len(self.stack) < 3:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        n = self.stack.pop()
        res = (a * b) % n if n != 0 else 0
        self.stack.append(res)

    def exp(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        res = pow(a, b) % (2**256)
        self.stack.append(res)

    def signextend(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        b = self.stack.pop()
        x = self.stack.pop()
        if b < 32:
            sign_bit = 1 << (8 * b - 1)
            x = x & ((1 << (8 * b)) - 1)
            if x & sign_bit:
                x = x | ~((1 << (8 * b)) - 1)
        self.stack.append(x)

    def lt(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(int(b < a))

    def gt(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(int(b > a))

    def slt(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(int(b < a))

    def sgt(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(int(b > a))

    def eq(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(int(a == b))

    def iszero(self):
        if len(self.stack) < 1:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        self.stack.append(int(a == 0))

    def and_op(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(a & b)

    def or_op(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(a | b)

    def xor_op(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(a ^ b)

    def not_op(self):
        if len(self.stack) < 1:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        self.stack.append(~a % (2**256))            # The result of the bitwise NOT operation needs to be modulo 2^256 to prevent overflow

    def byte_op(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        position = self.stack.pop()
        value = self.stack.pop()
        if position >= 32:
            res = 0
        else:
            res = (value // pow(256, 31 - position)) & 0xFF
        self.stack.append(res)

    def shl(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append((b << a) % (2**256))      # The result of the left shift operation needs to be modulo 2^256

    def shr(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(b >> a)                   # Right shift operation

    def sar(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        a = self.stack.pop()
        b = self.stack.pop()
        self.stack.append(b >> a)

    def mstore(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        offset = self.stack.pop()
        value = self.stack.pop()
        while len(self.memory) < offset + 32:
            self.memory.append(0)                   # Memory Expansion
        self.memory[offset:offset+32] = value.to_bytes(32, 'big')

    def mstore8(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        offset = self.stack.pop()
        value = self.stack.pop()
        while len(self.memory) < offset + 32:
            self.memory.append(0)                   # memory expansion
        self.memory[offset] = value & 0xFF          # take the least significant byte

    def sload(self):
        if len(self.stack) < 1:
            raise Exception('Stack underflow')
        key = self.stack.pop()
        value = self.storage.get(key, 0)        # If the key does not exist, return 0
        self.stack.append(value)

    def sstore(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        key = self.stack.pop()
        value = self.stack.pop()
        self.storage[key] = value

    def jump(self):
        if len(self.stack) < 1:
            raise Exception('Stack underflow')
        destination = self.stack.pop()
        if destination not in self.validJumpDest:
            raise Exception('Invalid jump destination')
        else:  self.pc = destination

    def jumpi(self):
        if len(self.stack) < 2:
            raise Exception('Stack underflow')
        destination = self.stack.pop()
        condition = self.stack.pop()
        if condition != 0:
            if destination not in self.validJumpDest:
                raise Exception('Invalid jump destination')
            else:  self.pc = destination

    def pc(self):
        self.stack.append(self.pc)

    def msize(self):
        self.stack.append(len(self.memory))

    def jumpdest(self):
        pass

    def run(self):
        self.findValidJumpDestinations()

        while self.pc < len(self.code):
            op = self.next_instruction()

            if PUSH1 <= op <= PUSH32:
                size = op - PUSH1 + 1
                self.push(size)
            elif op == PUSH0:
                self.stack.append(0)
            elif op == POP:
                self.pop()
            elif op == ADD:
                self.add()
            elif op == MUL:
                self.mul()
            elif op == SUB:
                self.sub()
            elif op == DIV:
                self.div()
            elif op == SDIV:
                self.sdiv()
            elif op == MOD:
                self.mod()
            elif op == SMOD:
                self.smod()
            elif op == ADDMOD:
                self.addmod()
            elif op == MULMOD:
                self.mulmod()
            elif op == EXP:
                self.exp()
            elif op == SIGNEXTEND:
                self.signextend()
            elif op == LT:
                self.lt()
            elif op == GT:
                self.gt()
            elif op == SLT:
                self.slt()
            elif op == SGT:
                self.sgt()
            elif op == EQ:
                self.eq()
            elif op == ISZERO:
                self.iszero()
            elif op == AND:
                self.and_op()
            elif op == OR:
                self.or_op()
            elif op == XOR:
                self.xor_op()
            elif op == NOT:
                self.not_op()
            elif op == BYTE:
                self.byte_op()
            elif op == SHL:
                self.shl()
            elif op == SHR:
                self.shr()
            elif op == SAR:
                self.sar()
            elif op == MLOAD:
                self.mload()
            elif op == MSTORE:
                self.mstore()
            elif op == MSTORE8:
                self.mstore8()
            elif op == SLOAD: 
                self.sload()
            elif op == SSTORE:
                self.sstore()
            elif op == MSIZE:
                self.msize()
            elif op == JUMP: 
                self.jump()
            elif op == JUMPDEST: 
                self.jumpdest()
            elif op == JUMPI: 
                self.jumpi()
            elif op == STOP:
                print('Program has been stopped')
                break
            elif op == PC:
                self.pc()
            else:
                raise Exception('Invalid opcode')

In [2]:
# STOP
code = b"\x00"
evm = EVM(code)
evm.run()
# output: Program has been stopped

Program has been stopped


In [ ]:
# JUMP
code = b"\x60\x04\x56\x00\x5b"
evm = EVM(code)
evm.run()
print(evm.pc)  
# output: 5, The program was not interrupted

5


In [ ]:
# JUMPI
code = b"\x60\x01\x60\x06\x57\x00\x5b"
evm = EVM(code)
evm.run()
print(evm.pc)  
# output: 7, The program was not interrupted

7


In [5]:
# JUMPI (Invalid jump destination)
code = b"\x60\x01\x60\x06\x57\x60\x5b"
evm = EVM(code)
evm.run()
print(evm.pc)  
# Exception: Invalid jump destination

Exception: Invalid jump destination

In [6]:
# JUMP (Invalid jump destination)
code = b"\x60\x04\x56\x60\x5b\x60\xff"
evm = EVM(code)
evm.run()
# Exception: Invalid jump destination

Exception: Invalid jump destination